# Final bulk funcn

In [6]:
import os
import shutil
import re
from openpyxl import load_workbook
from datetime import datetime
from win32com import client
import num2words
import psutil
from time import sleep
import mysql.connector
import subprocess
import logging
import babel.numbers
import configparser
#new comment
# Configure logging to write to a file
#logging.basicConfig(filename=r"C:\Users\AMARJEET\Desktop\python_bill\Log\BulkSell.txt", level=logging.INFO,
                    #format='%(asctime)s - %(levelname)s - %(message)s')
# Create a logger


log_directory = r"C:\Users\AMARJEET\Desktop\python_bill\Log\BulkSell.log"

# Check if the directory exists, if not, create it
#if os.path.exists(log_directory):
   # os.remove(log_directory)
   # os.makedirs(log_directory)

# Create the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  # Set the logging level

# Create handlers
console_handler = logging.StreamHandler()  # To log messages on the console
file_handler = logging.FileHandler(log_directory)  # To save logs to a file

# Set the logging level for handlers
console_handler.setLevel(logging.DEBUG)
file_handler.setLevel(logging.DEBUG)

# Create formatters
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - Line: %(lineno)d %(message)s')

# Add formatters to handlers
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Add handlers to logger
#logger.addHandler(console_handler)
logger.addHandler(file_handler)




config = configparser.ConfigParser()
config_file_path = r"C:\Users\AMARJEET\Desktop\python_bill\Config\config.ini.txt"
config.read(config_file_path)

paths = config['PATHS']
db_config = config['DATABASE']


def get_db_connection():
    """Establishes a connection to the MySQL database."""
    try:
        connection = mysql.connector.connect(host=db_config['Host'],user=db_config['User'],password=db_config['Password'], database=db_config['Database'])
       
        logging.info("Connection to MySQL DB successful")
        return connection
    except Error as e:
        logging.error(f"The error '{e}' occurred")
        return None
    
    



def Generate_Bills():
    
   # try :
    ##Reading the Bulk Excel file
    Bulk_workbook = load_workbook(filename=paths['BULK_SOURCE'])

    logging.info("----------------------------configuration files loading------------------------------------------ ")

    source = paths['SELL_BILL_FORMATE']
    destination= paths['BULK_SELL_BILL_EXCEL_PATH']
    file_loc=paths['CUSTOMER_DETAILS']
    pur_destination=paths['BULK_PURCHASE_BILL_PATH']

    logging.info(f"sell_bill_formate path ={source}")
    logging.info(f"destination bill path ={destination}")
    logging.info(f"customer_file loc ={file_loc}")
    logging.info(f"Purchase bill path ={pur_destination}")
    logging.info("-----------------------------------------------------------------------------------------------------")

    #logging.info("config file loaded ....")



    Bulk_sheet = Bulk_workbook.active
    Bulk_line =1
    while(True):
        Bulk_line=Bulk_line+1
        #Reading the file 
        try:
                cell_str="A"+str(Bulk_line)
                Current_Bill_No=int(Bulk_sheet[cell_str].value)
                cell_str="B"+str(Bulk_line)
                Current_Bill_Date=str(Bulk_sheet[cell_str].value).split()[0]
                cell_str="C"+str(Bulk_line)
                Party=str(Bulk_sheet[cell_str].value)
                cell_str="D"+str(Bulk_line)
                Bulk_GST=int(Bulk_sheet[cell_str].value)
                cell_str="E"+str(Bulk_line)
                Bulk_Amount=int(Bulk_sheet[cell_str].value)
                logging.info(f"----------------Invoice details----------------------------")
                logging.info(f"Current_Bill_No = {Current_Bill_No}")
                logging.info(f"Current_Bill_No = {Current_Bill_No}")
                logging.info(f"Current_Bill_Date ={Current_Bill_Date}")
                logging.info(f"Current_Bill_Date ={Current_Bill_Date}")
                logging.info(f"Party ={Party}")
                logging.info(f"Party ={Party}")
                logging.info(f"Bulk_GST ={Bulk_GST}")
                logging.info(f"Bulk_GST ={Bulk_GST}")
                logging.info(f" Bulk_Amount={Bulk_Amount}")
                logging.info(f" Bulk_Amount={Bulk_Amount}")

                logging.info(f"------------------------------------------------------------")
                print("generating bills: ",Current_Bill_No)




        except Exception as Ex:
            logging.info(f"Exception Occured during reading discription:{Ex}")
            logging.exception(f"Exception Occured during reading discription:{Ex}")
            break
        #Getting the values from DB

        sqlConn=get_db_connection()
        #logging.info(f"{sqlConn}")
        Balance_Amount =Bulk_Amount

        mycursor = sqlConn.cursor(buffered=True)
        #We can Optimize to reduce the Sql call to only one! in future ..
        if (Bulk_GST == 0):
            logging.info("in general selection")

            SelStock = "SELECT * FROM STOCK WHERE ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s  ORDER BY AMOUNT ASC"
            WhereStock = (Balance_Amount,)
            mycursor.execute(SelStock, WhereStock)
        else:
            SelStock = "SELECT * FROM STOCK WHERE GST = %s AND ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s  ORDER BY AMOUNT DESC"
            WhereStock = (Bulk_GST,Balance_Amount,)
            mycursor.execute(SelStock, WhereStock)
        ResStock = mycursor.fetchone()
        pur_invoice_no=ResStock[0]
        Cursor_Count=0
        #logging.info ("Select statement complete")
        #logging.info(f"first element selected is{ResStock[0]} ")
        ## Main logic should be flow from here ..
        #logging.info("Inside Main Logic ...")
        invoice_no=str(Current_Bill_No)
        pat = re.compile(r"[0-9]+")
        # Checks whether the whole string matches the re.pattern or not
        if re.fullmatch(pat, invoice_no):
            res=True
        else:
            logging.info("not valid invoice number")
            break;

        destination= paths['BULK_SELL_BILL_EXCEL_PATH']+invoice_no+".xlsx"


        dest = shutil.copyfile(source, destination)

    #load excel file
        workbook = load_workbook(filename=destination)

        #open workbook
        sheet = workbook.active

        cos_file=open(file_loc)
        cus_details=cos_file.readlines()
        cus_find=False
        while cus_find==False:
            name=Party
            list_len=len(cus_details)
            for i in cus_details:
                if (i.split('>')[0]==name):
                    add=i.split('>')[1]
                    cus_find=True
        #logging.info("customer details taken ....")

        res =False
        while res==False:
            logging.info("............Item Entry.........................................................")

            date=Current_Bill_Date
            res=True
            #
            #format = "%d-%m-%Y"
            #try:
             #   res = bool(datetime.strptime(date, format))
           # except ValueError:
            #    res = False
        #logging.info("date validation done ...")
        sheet["H3"]=date
        sheet["H5"]=invoice_no
        cell_no=13
        add_list=add.split("|")
        for i in add_list:
            cell_str="B"+str(cell_no)
            sheet[cell_str]=i
            cell_str="E"+str(cell_no)
            sheet[cell_str]=i
            cell_no=cell_no+1

        sum_total=0
        status=False
        discription_details=[]
        HSN_dis={}
        GST_HSN={}
        pur_invoice_no=ResStock[0]#To get Invoice result from Sql Query!
        pur_destination=paths['BULK_PURCHASE_BILL_PATH']+pur_invoice_no+".xlsx"

            #load excel file
        workbook_pur = load_workbook(filename=pur_destination,read_only=True) 
        #open workbook
        sheet_pur = workbook_pur.active
        logging.info(f"{sheet_pur}")
        net_amount=Bulk_Amount
        net_amount_temp=0

        Balance_Amount =net_amount
        pur_bill_dict= []

        while status ==False:
            if Cursor_Count == 0:
                change_bill =0
            else:
                change_bill = 1
            #logging.info("___________________________________________________________")
            logging.info(type(Cursor_Count))
            logging.info(f"Cursor_Count{Cursor_Count} ")
            logging.info(f"Chnage_Bill={change_bill}")
            #logging.info("_______________________________________________________________")

            if change_bill==1:
               # workbook_pur.save(filename=pur_destination)
                workbook_pur.close()
                if Cursor_Count < 4 :
                    if (Bulk_GST == 0):
                        logging.info(f"executing sql ")

                        SelStock = "SELECT * FROM STOCK WHERE ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s  ORDER  BY AMOUNT DESC"
                        WhereStock = (Balance_Amount,)
                    else:
                        logging.info(f"executing sql ")
                        SelStock = "SELECT * FROM STOCK WHERE GST = %s AND ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s ORDER BY AMOUNT DESC"
                        WhereStock = (Bulk_GST,Balance_Amount,)

                    mycursor.execute(SelStock, WhereStock)


                    ResStock = mycursor.fetchone()
                    pur_invoice_no=ResStock[0]
                    
                    


                else:#This case is to be handled in other cases also ...
                    if (Bulk_GST == 0):

                        SelStock = "SELECT * FROM STOCK WHERE ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s AND AMOUNT > %s  ORDER BY AMOUNT DESC"
                        WhereStock = (Balance_Amount,Balance_Amount,)
                    else:
                        SelStock = "SELECT * FROM STOCK WHERE GST = %s AND ((AMOUNT +(AMOUNT*GST*0.01))/QTY_IN_STOCK) < %s AND Amount >%s  ORDER BY AMOUNT DESC"
                        WhereStock = (Bulk_GST,Balance_Amount,Balance_Amount,)

                    mycursor.execute(SelStock, WhereStock)


                    ResStock = mycursor.fetchone()
                    pur_invoice_no=ResStock[0]
                    #logging.info(f"pur_invoice_no {pur_invoice_no}")




                pur_destination= paths['BULK_PURCHASE_BILL_PATH']+pur_invoice_no+".xlsx"

            #load excel file
                workbook_pur = load_workbook(filename=pur_destination,read_only=True)

            #open workbook
                sheet_pur = workbook_pur.active

            temp_list=[]
            pur_sl=ResStock[2]
            cell_str="B"+str(5+int(pur_sl))
            logging.info(f"Bill No :{pur_invoice_no} ,sl No :{pur_sl}")


            try:
                discription=sheet_pur[cell_str].value
                #logging.info("product_discription:",discription)
            except Exception as Ex:
                logging.info("Exception Occured during reading discription:",Ex)


            temp_list.append(discription)

            cell_str="G"+str(5+int(pur_sl))
            hsn=int(sheet_pur[cell_str].value)
            temp_list.append(hsn)
            if hsn in HSN_dis.keys():
                cc="sdddddd"
            else:
                HSN_dis[hsn]=0

            cell_str="V"+str(5+int(pur_sl))
            qty_rem=int(sheet_pur[cell_str].value)
            #net_amount=int(input("enter the amount"))
            if net_amount_temp ==0:
                net_amount_temp = net_amount
            if qty_rem ==0:
                logging.info("this item is not in stock")
                Cursor_Count = Cursor_Count +1
                continue
            #logging.info("quantity in stock for this product is",qty_rem)

            cell_str="I"+str(5+int(pur_sl))
            rate=round(float(sheet_pur[cell_str].value),2)


            cell_str="J"+str(5+int(pur_sl))
            dis=round(float(sheet_pur[cell_str].value),2)

            cell_str="S"+str(5+int(pur_sl))
            gst=int(sheet_pur[cell_str].value)
            logging.info(f"the value of gst is{gst}")
            temp_var =gst+100
            gst_factor =round(float(temp_var/100),2)
            logging.info(f"the gst_factor is :{gst_factor}")
            tax_amount =net_amount_temp/gst_factor
            logging.info(f"taxable ammount is{tax_amount}")


            rate=round((rate+(0.032*rate)),2)
            logging.info(f'genral rate of the product is:{rate}')
            rate_with_dis=round((rate-(rate*dis/100)),2)
            req_qnt=int((tax_amount/rate_with_dis))
            logging.info(f"tax_amount={tax_amount} ratewithdis={rate_with_dis}")
            logging.info(f"req_qnt :{req_qnt},qty_rem: {qty_rem}")

            if req_qnt >= qty_rem:
               # Temp_ammount = Temp_ammount +(req_qnt*rate_with_dis)+(req_qnt*rate_with_dis*gst/100)
                logging.info(f"NOT SUFFICIENT NEED TO ADD MORE")
                rate = rate_with_dis 
                req_qnt = qty_rem
                logging.info(f"pur_invoice_no ={pur_invoice_no}, pur_sl ={pur_sl}")
                #logging.info ("sl_no_type =",type(pur_sl))
                sql="DELETE FROM STOCK WHERE BILL_NO =%s AND BILL_SERIAL_NO =%s"
                val = (pur_invoice_no,pur_sl)
                mycursor.execute(sql, val)

                sqlConn.commit()

                #logging.info("{mycursor.rowcount}record(s) affected")

               # break

            else:
                rate=round((tax_amount/req_qnt),3)
                print("req_qnt = ",req_qnt,"pur_invoice_no = ",pur_invoice_no ,"pur_sl ",pur_sl)
                sql = "UPDATE STOCK SET QTY_IN_STOCK = (QTY_IN_STOCK - %s) ,DIRTY_FLG =1 ,AMOUNT = ((QTY_IN_STOCK - %s)*((AMOUNT -(AMOUNT*GST*0.01))/QTY_IN_STOCK)) WHERE   BILL_NO =%s  AND BILL_SERIAL_NO =%s"
                val = (req_qnt,req_qnt,pur_invoice_no,pur_sl,)

                mycursor.execute(sql, val)

                sqlConn.commit()

               # logging.info(mycursor.rowcount, "record(s) affected")




            dis=0
            quantity= req_qnt
            temp_quantity=req_qnt


            temp_list.append(temp_quantity)
            temp_list.append(rate)
            logging.info(f"optimum rate for product is{rate} ")
            quantity=temp_quantity
            HSN_dis[hsn]=HSN_dis[hsn]+((quantity*rate)-(quantity*rate*dis/100))
            temp_list.append(dis)
            temp_list.append((quantity*rate)-(quantity*rate*dis/100))
            amount=(quantity*rate)-(quantity*rate*dis/100)


            cell_str="S"+str(5+int(pur_sl))
            gst=int(sheet_pur[cell_str].value)
            if hsn in GST_HSN.keys():
                cc="sdddddd"
            else:
                GST_HSN[hsn]=gst

            #gst=int(input("GST_rate"))
            sum_total=sum_total+(quantity*rate)+(quantity*rate*gst/100)
            temp_list.append(round((quantity*rate)+(quantity*rate*gst/100),2))
            discription_details.append(temp_list)
            logging.info(f"total amount till now incluing gst{sum_total}")
            logging.info(f"NEED TO ADD MORE OF RS {(net_amount -sum_total)}")
            Cursor_Count =Cursor_Count +1
            Balance_Amount=net_amount -sum_total

            net_amount_temp = net_amount -sum_total
            #inserting sell bill_no
            #logging.info("going to save entries in purchase bills")
            cell_str="L"+str(5+int(pur_sl))
            pur_bill_entry={}
            pur_bill_entry['PURCHASE_BILL']=pur_invoice_no
            pur_bill_entry['PURCHASE_BILL_SL']=pur_sl
            pur_bill_entry['SELL_BILL_NO']=invoice_no
            pur_bill_entry['QUANTITY']=quantity
            pur_bill_entry['AMMOUNT']=amount
            pur_bill_dict.append(pur_bill_entry)





            if(Balance_Amount >1):
                status=False
                #Curson_Count=Cursor_Count+1
            else:
                status=True
        num=21
        count=0
        s_t=0
        logging.info(discription_details)
        for i in discription_details:
            count=count+1
            cell_str="A"+str(num)
            sheet[cell_str]=count
            cell_str="B"+str(num)
            sheet[cell_str]=i[0]
            cell_str="E"+str(num)
            sheet[cell_str]=i[1]
            cell_str="F"+str(num)
            sheet[cell_str]=i[2]
            cell_str="G"+str(num)
            sheet[cell_str]=i[3]
            cell_str="H"+str(num)
            sheet[cell_str]=i[4]
            cell_str="I"+str(num)
            sheet[cell_str]=i[5]
            s_t=s_t+i[-2]
            num=num+1

        sheet["H27"]=s_t


        count=37
        tax_sum=0
        for key, value_1 in HSN_dis.items():

            cell_str="B"+str(count)
            sheet[cell_str]=key
            sheet.cell(count,3).value=value_1
        #cell_str="D"+str(count)
        #sheet[cell_str]=value_1
            cell_str="E"+str(count)
            sheet[cell_str]=(GST_HSN[key]/2)/100
            cell_str="F"+str(count)
            sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
            cell_str="G"+str(count)
            sheet[cell_str]=(GST_HSN[key]/2)/100
            cell_str="H"+str(count)
            sheet[cell_str]=(GST_HSN[key]/2)*value_1/100
            cell_str="I"+str(count)
            sheet[cell_str]=(GST_HSN[key])*value_1/100
            tax_sum=tax_sum+(GST_HSN[key])*value_1/100
            count=count+1
        sheet["J41"]=tax_sum
        sheet["C42"]=num2words.num2words(round(tax_sum,2), lang='en_IN')+"ONLY"
        sheet["H28"]=tax_sum/2
        sheet["H29"]=tax_sum/2
        sheet["H30"]=s_t+tax_sum


        sum_t=s_t+tax_sum
        r_total=round(sum_t,0)
        sheet['H31']=net_amount-(s_t+tax_sum)
        sheet['H32']=net_amount
        sheet["C34"]=num2words.num2words(net_amount, lang='en_IN')+"ONLY"

    #save the file
        #workbook_pur.save(filename=pur_destination)
        workbook.save(filename=destination)
        workbook.close()
        workbook_pur.close()
        print (pur_bill_dict)
        #subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])

        #SAVE THE CHANGES IN PURCHASE_BILL 
        for pur_bill_entry in pur_bill_dict:


            pur_invoice_no=pur_bill_entry['PURCHASE_BILL']
            pur_sl=pur_bill_entry['PURCHASE_BILL_SL']
            #invoice_no=pur_bill_entry['SELL_BILL_NO']
            quantity=pur_bill_entry['QUANTITY']
            amount=pur_bill_entry['AMMOUNT']

            pur_destination= paths['BULK_PURCHASE_BILL_PATH']+pur_invoice_no+".xlsx"
            print(pur_destination)

            #load excel file
            workbook_pur = load_workbook(filename=pur_destination)

            #open workbook
            sheet_pur = workbook_pur.active
            logging.info("pur vhase bill opned ")

            #inserting sell bill_no
            cell_str="L"+str(5+int(pur_sl))
            try:
                sell_bill_no=sheet_pur[cell_str].value
                sell_bill_no=sell_bill_no+'|'+str(invoice_no)
            except TypeError:
                sell_bill_no=str(invoice_no)
            sheet_pur[cell_str]=sell_bill_no
            #inserting sell_qty_wrt_bill
            cell_str="M"+str(5+int(pur_sl))
            try:
                sell_qty_no=sheet_pur[cell_str].value
                sell_qty_no=sell_qty_no+'|'+str(quantity)
            except TypeError:
                sell_qty_no=str(quantity)
            sheet_pur[cell_str]=sell_qty_no


            #inserting total_quantity_selled in audit

            cell_str="O"+str(5+int(pur_sl))
            try:
                total_selled_qntity=int(sheet_pur[cell_str].value)
            except TypeError:
                total_selled_qntity=0;

            total_selled_qntity=total_selled_qntity+quantity
            sheet_pur[cell_str]=str(total_selled_qntity)

            #inserting total selled qntity
            cell_str="O"+str(5+int(pur_sl))
            sheet_pur[cell_str]=total_selled_qntity
            cell_str="U"+str(5+int(pur_sl))
            sheet_pur[cell_str]=total_selled_qntity


            #inserting quantity in stock
            cell_str="H"+str(5+int(pur_sl))
            pur_total_qntity=int(sheet_pur[cell_str].value)
            stock_qnty=pur_total_qntity-total_selled_qntity
            cell_str="V"+str(5+int(pur_sl))
            sheet_pur[cell_str]=stock_qnty

            #inserting sell amount

            cell_str="X"+str(5+int(pur_sl))
            try:
                total_selled_amount=int(sheet_pur[cell_str].value)
            except TypeError:
                total_selled_amount=0
            total_selled_amount=total_selled_amount+amount
            sheet_pur[cell_str]=total_selled_amount

            cell_str="P"+str(5+int(pur_sl))
            sheet_pur[cell_str]= round( (float(total_selled_amount)/float(total_selled_qntity)),2)

            #inserting gst__received

            cell_str="Z"+str(5+int(pur_sl))
            try:
                gst_amount=float(round(sheet_pur[cell_str].value))
            except TypeError:
                gst_amount=0
            gst_amount=gst_amount+round((amount*gst/100),2)
            sheet_pur[cell_str]=str(gst_amount)

            #inserting gst__balance


            cell_str="Y"+str(5+int(pur_sl))
            try:
                gst_recv=int(sheet_pur[cell_str].value)
            except TypeError:
                gst_recv=0
            gst_balance=gst_amount-gst_recv
            cell_str="AA"+str(5+int(pur_sl))
            sheet_pur[cell_str]=str(gst_balance)
            workbook_pur.save(filename=pur_destination)
    
            workbook_pur.close()

        #pdf_path=r"C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\PDF\sk"+invoice_no
        pdf_path1=paths['SELL_BILL_PDF_PATH']+invoice_no
    # Open  Excel
        subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
        excel = client.Dispatch("Excel.Application")
        excel.Visible = False

        # Read Excel File
        print(destination)
        sheets = excel.Workbooks.Open(destination)
        work_sheets = sheets.Worksheets[0]

        # Convert into PDF File
        #work_sheets.ExportAsFixedFormat(0, pdf_path)
        work_sheets.ExportAsFixedFormat(0, pdf_path1)
        ##############COMMENTED FOR DISABLE logging.infoING ###########################################
       # for i in range(1):    
        #    os.startfile(destination, "logging.info")
         #   for p in psutil.process_iter(): #Close Acrobat after logging.infoing the PDF
          #      if 'Excel' in str(p):
           #         p.kill()
   # except Exception as ex:
    #    logging.error(f"Exception ocuured {ex}")
     #  file_handler.close()
      #  return 0





















        #logging.info need to be commented ..
       
        
#Update_Stock()        
Generate_Bills()
subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
file_handler.close()
                
            
        
    





generating bills:  413
req_qnt =  106 pur_invoice_no =  EKTA4159 pur_sl  1
[{'PURCHASE_BILL': 'ISHA 2441', 'PURCHASE_BILL_SL': 4, 'SELL_BILL_NO': '413', 'QUANTITY': 30, 'AMMOUNT': 491.99999999999994}, {'PURCHASE_BILL': '21222', 'PURCHASE_BILL_SL': 10, 'SELL_BILL_NO': '413', 'QUANTITY': 131, 'AMMOUNT': 762.4200000000001}, {'PURCHASE_BILL': 'EKTA3773', 'PURCHASE_BILL_SL': 24, 'SELL_BILL_NO': '413', 'QUANTITY': 60, 'AMMOUNT': 756.0}, {'PURCHASE_BILL': 'EKTA4159', 'PURCHASE_BILL_SL': 1, 'SELL_BILL_NO': '413', 'QUANTITY': 106, 'AMMOUNT': 540.812}]
C:\Users\AMARJEET\Desktop\python_bill\AUDIT\AuditISHA 2441.xlsx
C:\Users\AMARJEET\Desktop\python_bill\AUDIT\Audit21222.xlsx
C:\Users\AMARJEET\Desktop\python_bill\AUDIT\AuditEKTA3773.xlsx
C:\Users\AMARJEET\Desktop\python_bill\AUDIT\AuditEKTA4159.xlsx
C:\Users\AMARJEET\Desktop\python_bill\SELL_BILL\EXCEL\sk413.xlsx
generating bills:  414
req_qnt =  198 pur_invoice_no =  EKTA3413  pur_sl  8
[{'PURCHASE_BILL': 'EKTA4159', 'PURCHASE_BILL_SL': 1, 'SELL_